In [427]:
import pandas as pd
import os
from datetime import date, timedelta, datetime
from metric_helper import read_helper
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [429]:
datetime.today().strftime('%Y-%m-%d %H:%M:%S')

'2025-11-17 10:48:01'

## Load Data

In [390]:
def read_recommendations(dt_list):
    
    df = pd.DataFrame()
    
    for dt in dt_list:
        
        df_temp = pd.DataFrame()
    
        if os.path.exists(f'./results/execution/default/zips_to_remediate/{dt}/'):
            df_rem = pd.read_parquet(f'./results/execution/default/zips_to_remediate/{dt}/dea_th_0.9_fc_switch_0.15_rec_count_999999.parquet')
            df_temp = pd.concat([df_temp, df_rem])

        if os.path.exists(f'./results/execution/default/zips_to_expand/{dt}/'):
            df_exp = pd.read_parquet(f'./results/execution/default/zips_to_expand/{dt}/dea_th_0.9_fc_switch_0.15_rec_count_999999.parquet')
            df_temp = pd.concat([df_temp, df_exp])
            
        df_temp['run_date'] = dt
        
        df = pd.concat([df_temp, df])
        
        
    return df

In [391]:
def read_metadata(dt_list):
    
    df = pd.DataFrame()
    
    for dt in dt_list:
    
        if os.path.exists(f'./results/execution/default/metadata/{dt}/zips_to_recommend.parquet'):
            df_temp = pd.read_parquet(f'./results/execution/default/metadata/{dt}/zips_to_recommend.parquet')
            df_temp['run_date'] = dt
            df = pd.concat([df, df_temp])
        
        
    return df

In [392]:
dt_list = ['2025-11-13'] # '2025-11-10','2025-11-11','2025-11-12','2025-11-13'

df = read_recommendations(dt_list) # recommendations
df_m = read_metadata(dt_list) # metadata

In [393]:
df.head(2)

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,fc_switch_package_per,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,FDXHD_days_behind,ONTRGD_days_behind,run_date
0,95531,1,28.285714,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.749669,0.000000,148.0,53.0,0.993243,0.433962,0.027663,0.05,2025-11-13
1,60517,1,32.142857,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.488105,0.022222,223.0,32.0,0.982063,0.593750,0.075597,0.27,2025-11-13


In [394]:
df_m.head(2)

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
4,75209,1,0.022599,25.285714,0.987654,1.0,ok,162.0,12.0,0.070422,0.04,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
7,75214,1,0.017677,56.571429,0.994911,1.0,ok,393.0,8.0,0.052681,0.04,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [395]:
df = df.fillna('None')
df_m = df_m.fillna('None')

## Recommendation Distribution

In [396]:
df.groupby(['run_date','active','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,final_recommendation,zip5
0,2025-11-13,1,deactivate,136


In [325]:
df.groupby(['run_date','active','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,deactivate,None,deactivate,136


In [326]:
df_m.groupby(['run_date','active','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,None,None,None,7
1,2025-11-13,1,deactivate,None,deactivate,136
2,2025-11-13,1,ok,None,None,3245


In [327]:
df_m.groupby(['run_date','active','reason_code_dea','reason_code_backlog','current_recommendation','last_recommendation','final_recommendation'])['zip5'].nunique().reset_index()

,run_date,active,reason_code_dea,reason_code_backlog,current_recommendation,last_recommendation,final_recommendation,zip5
0,2025-11-13,1,None,None,None,None,None,7
1,2025-11-13,1,None,ok,ok,None,None,142
2,2025-11-13,1,deactivate,None,deactivate,None,deactivate,15
3,2025-11-13,1,deactivate,ok,deactivate,None,deactivate,121
4,2025-11-13,1,ok,None,ok,None,None,295
5,2025-11-13,1,ok,ok,ok,None,None,2808


## Zip by Zip Analysis

In [328]:
df = df.sort_values(['zip5','run_date'])
df_m = df_m.sort_values(['zip5','run_date'])

In [329]:
df['count'] = df.groupby('zip5')['run_date'].transform('nunique')

In [330]:
df.loc[df['count'] == 1].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count
3824,07006,1,45.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.403051,1,2025-11-13,1
8187,07042,1,51.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.049946,1,2025-11-13,1
4111,07044,1,29.285714,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.150590,1,2025-11-13,1
8192,07059,1,28.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.089012,1,2025-11-13,1
8312,07081,1,29.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.150590,1,2025-11-13,1


In [331]:
df_m.loc[df_m['zip5'] == '07066'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
4117,07066,1,0.0,30.571429,1.0,0.980392,ok,76.0,102.0,0.046379,0.07,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [332]:
df.loc[df['count'] == 2].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count


In [333]:
df_m.loc[df_m['zip5'] == '07042'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
8187,07042,1,0.002762,51.714286,0.992208,0.88,deactivate,385.0,25.0,0.0424,0.1,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13


In [334]:
df.loc[df['count'] == 3].head()

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count


In [335]:
df_m.loc[df_m['zip5'] == '07950'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
1584,07950,1,0.0,38.428571,0.996296,0.833333,deactivate,270.0,12.0,0.04838,0.1,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13


## Particular Zip Analysis

### NA DEA zips

In [336]:
dea = read_helper(
    './data/execution_data/unpadded_dea',
    start_date='2025-10-31',
    end_date='2025-11-11',
    date_col_name='unpadded_edd',
    cols=['unpadded_edd',
        'ffmcenter_name',
        'carrier_code',
        'zip5',
        'package_count',
        'unpadded_edd_dea_count'
        ]
    )
dea.head(2)

,unpadded_edd,ffmcenter_name,carrier_code,zip5,package_count,unpadded_edd_dea_count
0,2025-10-31,AVP1,FDXHD,01225,3,3
1,2025-10-31,AVP1,FDXHD,01267,3,3


In [337]:
dea = dea.groupby(
    ['zip5','unpadded_edd', 'carrier_code']).agg(
        {'package_count':'sum', 'unpadded_edd_dea_count':'sum'}).reset_index()
dea = dea.sort_values(['zip5','unpadded_edd'])

In [338]:
dea.groupby(['carrier_code','unpadded_edd'])['zip5'].nunique()

carrier_code  unpadded_edd
FDXHD         2025-10-31        542
              2025-11-01      23225
              2025-11-02       8675
              2025-11-03      25181
              2025-11-04      24421
              2025-11-05      25450
              2025-11-06      25641
              2025-11-07      25288
              2025-11-08      23210
              2025-11-09       8768
              2025-11-10      25085
              2025-11-11      24273
ONTRGD        2025-10-31          7
              2025-11-01       4179
              2025-11-02       5360
              2025-11-03       6164
              2025-11-04       6038
              2025-11-05       5893
              2025-11-06       5715
              2025-11-07       5457
              2025-11-08       4414
              2025-11-09       5389
              2025-11-10       6178
              2025-11-11       6081
Name: zip5, dtype: int64

In [339]:
df_m['zip5'].nunique()

3388

In [340]:
df_m.loc[df_m['FDXHD_unpadded_edd_dea'] != 'None']['zip5'].nunique()

3388

In [341]:
df_m.loc[df_m['FDXHD_unpadded_edd_dea'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [342]:
dea.loc[dea['zip5'].isin(df_m.loc[df_m['FDXHD_unpadded_edd_dea'] == 'None']['zip5'])].head()

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count


In [343]:
df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] != 'None']['zip5'].nunique()

3248

In [344]:
df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
3994,30121,1,0.015306,28.000000,0.966102,None,None,177.0,None,0.060738,0.04,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
2269,32903,1,0.014925,28.714286,0.995074,None,None,203.0,None,0.05057,0.04,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
1054,33030,1,0.005348,26.714286,0.989848,None,None,197.0,None,0.053662,0.04,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
8402,33415,1,0.005495,26.000000,0.983784,None,None,185.0,None,0.050069,0.03,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13
2438,33781,1,0.013333,32.142857,0.973077,None,None,260.0,None,0.044981,0.03,ok,ok,None,None,ok,None,None,None,None,None,None,None,None,None,2025-11-13


In [345]:
dea.loc[dea['zip5'].isin(df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] == 'None']['zip5'])].sort_values(['zip5','unpadded_edd'])[0:100]

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count
98300,30121,2025-11-01,FDXHD,21,21
98301,30121,2025-11-02,FDXHD,29,29
98302,30121,2025-11-02,ONTRGD,2,1
98303,30121,2025-11-03,FDXHD,38,38
98304,30121,2025-11-04,FDXHD,31,31
98305,30121,2025-11-05,FDXHD,20,20
98306,30121,2025-11-06,FDXHD,29,25
98307,30121,2025-11-06,ONTRGD,1,1
98308,30121,2025-11-07,FDXHD,25,25
98309,30121,2025-11-08,FDXHD,37,37


In [346]:
df_m.loc[df_m['zip5'] == '07444'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [347]:
dea.loc[dea['zip5'] == '07444']

,zip5,unpadded_edd,carrier_code,package_count,unpadded_edd_dea_count
23089,07444,2025-11-01,FDXHD,21,21
23090,07444,2025-11-02,FDXHD,29,29
23091,07444,2025-11-03,FDXHD,26,26
23092,07444,2025-11-03,ONTRGD,1,1
23093,07444,2025-11-04,FDXHD,23,22
23094,07444,2025-11-05,FDXHD,15,15
23095,07444,2025-11-06,FDXHD,19,18
23096,07444,2025-11-06,ONTRGD,2,2
23097,07444,2025-11-07,FDXHD,15,15
23098,07444,2025-11-08,FDXHD,23,23


### NA Backlog Zips

In [348]:
backlog_df = read_helper(
        './data/execution_data/backlog',
        start_date='2025-10-31',
        end_date='2025-11-13',
        date_col_name='date'
        )
backlog_df.head(2)

,dest_facility_num,zip5,days_behind,carrier_code,date
0,950,95641,0.039347,FDXHD,2025-10-31
1,402,47154,0.048078,FDXHD,2025-10-31


In [349]:
backlog_df.groupby(['carrier_code','date'])['zip5'].nunique()

carrier_code  date      
FDXHD         2025-10-31    86101
              2025-11-01    86178
              2025-11-02    86178
              2025-11-03    86178
              2025-11-04    86178
              2025-11-05    86114
              2025-11-07    86191
              2025-11-08    86114
              2025-11-09    86114
              2025-11-10    86114
              2025-11-11    86114
              2025-11-12    86114
              2025-11-13    86114
ONTRGD        2025-10-31    11373
              2025-11-03    11373
              2025-11-04    11373
              2025-11-05    11373
              2025-11-06    11373
              2025-11-07    11373
              2025-11-10    11373
              2025-11-11    11373
              2025-11-12    11373
              2025-11-13    11373
Name: zip5, dtype: int64

In [350]:
df_m['zip5'].nunique()

3388

In [351]:
df_m.loc[df_m['FDXHD_days_behind'] != 'None']['zip5'].nunique()

3071

In [352]:
df_m.loc[df_m['FDXHD_days_behind'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
4070,06002,1,0.000000,30.571429,1.000000,0.958904,ok,106.0,73.0,None,0.05,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
4075,06029,1,0.003817,37.428571,0.988095,0.97619,ok,168.0,84.0,None,0.05,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
4076,06032,1,0.007692,37.142857,1.000000,0.98,ok,128.0,100.0,None,0.05,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
4077,06033,1,0.014799,67.571429,0.992806,0.981707,ok,278.0,164.0,None,0.05,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
4079,06040,1,0.027273,62.857143,1.000000,0.984615,ok,202.0,130.0,None,0.05,None,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [353]:
backlog_df.loc[backlog_df['zip5'] == '77007']

,dest_facility_num,zip5,days_behind,carrier_code,date
81840,6030,77007,0.02,ONTRGD,2025-10-31
33109,6030,77007,0.02,ONTRGD,2025-11-03
88470,6030,77007,0.02,ONTRGD,2025-11-04
93015,6030,77007,0.03,ONTRGD,2025-11-05
9736,6030,77007,0.03,ONTRGD,2025-11-06
60772,6030,77007,0.06,ONTRGD,2025-11-07
44513,6030,77007,0.02,ONTRGD,2025-11-10
72429,6030,77007,0.01,ONTRGD,2025-11-11
50513,6030,77007,0.01,ONTRGD,2025-11-12
66964,6030,77007,0.02,ONTRGD,2025-11-13


In [354]:
df_m.loc[df_m['ONTRGD_days_behind'] != 'None']['zip5'].nunique()

3388

In [355]:
df_m.loc[df_m['ONTRGD_days_behind'] == 'None'].head()

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date


In [356]:
backlog_df

,dest_facility_num,zip5,days_behind,carrier_code,date
0,950,95641,0.039347,FDXHD,2025-10-31
1,402,47154,0.048078,FDXHD,2025-10-31
2,711,75694,0.089074,FDXHD,2025-10-31
3,994,98390,0.051329,FDXHD,2025-10-31
4,745,74599,0.080460,FDXHD,2025-10-31
...,...,...,...,...,...
97691,23,07508,0.100000,ONTRGD,2025-11-13
97692,23,07876,0.100000,ONTRGD,2025-11-13
97693,3420,96155,0.300000,ONTRGD,2025-11-13
97694,13,27344,0.040000,ONTRGD,2025-11-13


## Get Metadata to Final Rec

In [378]:
df.head(2)

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count
3824,07006,1,45.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.403051,1,2025-11-13,1
8187,07042,1,51.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.049946,1,2025-11-13,1


In [397]:
df_m.head(2)

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
4,75209,1,0.022599,25.285714,0.987654,1.0,ok,162.0,12.0,0.070422,0.04,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13
7,75214,1,0.017677,56.571429,0.994911,1.0,ok,393.0,8.0,0.052681,0.04,ok,ok,ok,None,None,None,None,None,None,None,None,None,None,None,2025-11-13


In [380]:
metadata_cols = ['zip5','run_date','fc_switch_package_per',
                 'FDXHD_act_package_count','ONTRGD_act_package_count',
                 'FDXHD_unpadded_edd_dea','ONTRGD_unpadded_edd_dea',
                 'FDXHD_days_behind','ONTRGD_days_behind',
                 'reason_code_dea','reason_code_backlog'
                 ]

In [381]:
df_c = df\
    .merge(
        df_m[metadata_cols], 
        on=['zip5','run_date'], 
        how='left'
        )

In [382]:
df_c = df_c.sort_values(['run_date','priority_score'], ascending=False)

In [383]:
df_c.head(10)

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,selected,run_date,count,fc_switch_package_per,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,FDXHD_days_behind,ONTRGD_days_behind,reason_code_dea,reason_code_backlog
134,95531,1,28.285714,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.749669,1,2025-11-13,1,0.000000,148.0,53.0,0.993243,0.433962,0.027663,0.05,deactivate,ok
80,60517,1,32.142857,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.488105,1,2025-11-13,1,0.022222,223.0,32.0,0.982063,0.59375,0.075597,0.27,deactivate,ok
87,60546,1,26.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.484882,1,2025-11-13,1,0.042781,162.0,27.0,0.962963,0.592593,0.075597,0.27,deactivate,ok
36,33437,1,42.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.417189,1,2025-11-13,1,0.003356,330.0,8.0,0.993939,0.625,0.050069,0.03,deactivate,ok
0,07006,1,45.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.403051,1,2025-11-13,1,0.012579,345.0,11.0,0.988406,0.636364,0.045958,0.10,deactivate,ok
135,98109,1,68.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.389336,1,2025-11-13,1,0.000000,191.0,305.0,0.942408,0.839344,0.042277,0.02,deactivate,ok
70,60440,1,33.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.364357,1,2025-11-13,1,0.051064,252.0,18.0,0.980159,0.666667,0.068483,0.27,deactivate,ok
66,60181,1,37.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.352331,1,2025-11-13,1,0.026616,220.0,42.0,0.995455,0.690476,0.059825,0.27,deactivate,ok
34,33189,1,26.000000,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.349456,1,2025-11-13,1,0.010989,187.0,3.0,0.962567,0.666667,0.053662,0.04,deactivate,ok
100,77586,1,34.857143,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.349456,1,2025-11-13,1,0.118852,283.0,3.0,0.971731,0.666667,0.07012,0.02,deactivate,ok


In [407]:
df_m['ONTRGD_unpadded_edd_dea'] = pd.to_numeric(df_m['ONTRGD_unpadded_edd_dea'], errors='coerce')

In [410]:
df_m.loc[df_m['ONTRGD_unpadded_edd_dea'] < 0.9]

,zip5,active,fc_switch_package_per,daily_package_count_avg,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,reason_code_dea,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_days_behind,ONTRGD_days_behind,reason_code_backlog,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,recommendation_date,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,run_date
128,30144,1,0.004640,61.571429,1.000000,0.875000,deactivate,430.0,16.0,0.060738,0.04,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
130,30157,1,0.014925,67.000000,0.971175,0.840000,deactivate,451.0,25.0,0.085605,0.04,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
247,60462,1,0.052632,38.000000,0.995833,0.842105,deactivate,240.0,57.0,0.075597,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
257,60477,1,0.029240,48.857143,0.983713,0.791667,deactivate,307.0,48.0,0.075597,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
261,60487,1,0.021739,32.857143,0.994792,0.789474,deactivate,192.0,38.0,0.075597,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
266,60504,1,0.030043,33.285714,0.990476,0.878788,deactivate,210.0,33.0,0.059825,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
398,60137,1,0.048257,53.285714,0.993548,0.800000,deactivate,310.0,55.0,0.090616,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
403,60148,1,0.036430,78.428571,0.997899,0.773196,deactivate,476.0,97.0,0.059825,0.27,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
509,43213,1,0.000000,25.714286,0.983696,0.857143,deactivate,184.0,7.0,0.069108,0.05,ok,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13
539,76148,1,0.015306,28.000000,0.986607,0.857143,deactivate,224.0,7.0,None,0.04,None,deactivate,deactivate,None,None,None,None,None,None,None,deactivate,deactivate,None,None,2025-11-13


In [411]:
df

,zip5,active,daily_package_count_avg,current_recommendation,current_reason_dea,current_reason_shutdown,current_reason_backlog,last_recommendation,last_reason_dea,last_reason_shutdown,last_reason_backlog,final_recommendation,final_reason_dea,final_reason_shutdown,final_reason_backlog,priority_score,fc_switch_package_per,FDXHD_act_package_count,ONTRGD_act_package_count,FDXHD_unpadded_edd_dea,ONTRGD_unpadded_edd_dea,FDXHD_days_behind,ONTRGD_days_behind,run_date
0,95531,1,28.285714,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.749669,0.000000,148.0,53.0,0.993243,0.433962,0.027663,0.05,2025-11-13
1,60517,1,32.142857,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.488105,0.022222,223.0,32.0,0.982063,0.593750,0.075597,0.27,2025-11-13
2,60546,1,26.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.484882,0.042781,162.0,27.0,0.962963,0.592593,0.075597,0.27,2025-11-13
3,33437,1,42.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.417189,0.003356,330.0,8.0,0.993939,0.625000,0.050069,0.03,2025-11-13
4,07006,1,45.428571,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.403051,0.012579,345.0,11.0,0.988406,0.636364,0.045958,0.10,2025-11-13
5,98109,1,68.714286,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.389336,0.000000,191.0,305.0,0.942408,0.839344,0.042277,0.02,2025-11-13
6,60440,1,33.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.364357,0.051064,252.0,18.0,0.980159,0.666667,0.068483,0.27,2025-11-13
7,60181,1,37.571429,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.352331,0.026616,220.0,42.0,0.995455,0.690476,0.059825,0.27,2025-11-13
8,77586,1,34.857143,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.349456,0.118852,283.0,3.0,0.971731,0.666667,0.07012,0.02,2025-11-13
9,33189,1,26.000000,deactivate,deactivate,None,None,None,None,None,None,deactivate,deactivate,None,None,0.349456,0.010989,187.0,3.0,0.962567,0.666667,0.053662,0.04,2025-11-13
